Using [Langchain RetrievalQA](https://github.com/langchain-ai/langchain/blob/master/cookbook/openai_functions_retrieval_qa.ipynb) to chat with our collection data.

In [38]:
!pip install --quiet langchain langchain-openai requests jq chromadb

In [26]:
import json
import jq
import requests

from langchain_community.document_loaders import JSONLoader


# First page of works on the ACMI Public API
json_data = requests.get('https://api.acmi.net.au/works/').json()
file_path = 'data.json'
with open(file_path, 'w', encoding='utf-8') as json_file:
    json.dump(json_data, json_file)
json_loader = JSONLoader(
    file_path=file_path,
    jq_schema='.results[]',
    text_content=False,
)
data = json_loader.load()
for i, item in enumerate(data):
    item.metadata['source'] = f'https://api.acmi.net.au/works/{json_data["results"][i]["id"]}'

In [13]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [48]:
# Run this step if you'd like langchain tracing via LangSmith https://smith.langchain.com
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

··········


In [27]:
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings


embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(data, embeddings)

In [28]:
from langchain.chains import RetrievalQA
from langchain.chains import create_qa_with_sources_chain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(temperature=0, model='gpt-4-turbo-preview')
qa_chain = create_qa_with_sources_chain(llm)
doc_prompt = PromptTemplate(
    template="Content: {page_content}\nSource: {source}",
    input_variables=["page_content", "source"],
)
final_qa_chain = StuffDocumentsChain(
    llm_chain=qa_chain,
    document_variable_name="context",
    document_prompt=doc_prompt,
)
retrieval_qa = RetrievalQA(
    retriever=docsearch.as_retriever(), combine_documents_chain=final_qa_chain
)

In [43]:
query = 'What can you tell me about "The Drover\'s Wife"? Who made it?'
response = retrieval_qa.run(query)
print(f'Answer: {json.loads(response)["answer"]}')
print(f'Sources: {json.loads(response)["sources"]}')

Answer: "The Drover's Wife" is a work that includes elements such as "The Drover's Wife: Molly Johnson (Leah Purcell) with clapper board" and "The Drover's Wife: Yadaka (Rob Collins) and Danny (Malachi Dower-Roberts) still." These works were created in Australia in 2021 and are part of the group "The Drover's Wife: The Legend of Molly Johnson." Tess Schofield is credited as the costume designer for the work featuring Molly Johnson, indicating her involvement in the production's design aspects. The works are credited to Oombarra Productions and Bunya Productions, as indicated by the credit line. They were displayed at ACMI: Gallery 1, with the last display date noted as August 14, 2024. The works are categorized under the curatorial section "The Story of the Moving Image → Moving Australia → MA-11. Screen Australia Showcase → MA-11-C01" and are identified as 2D objects and costume continuity elements/pictorial.
Sources: ['https://api.acmi.net.au/works/121889', 'https://api.acmi.net.au/w

In [46]:
query = 'What can you tell me about "Minyak Sawit Keluarga"? Who made it? Where was it on display?'
response = retrieval_qa.run(query)
print(f'Answer: {json.loads(response)["answer"]}')
print(f'Sources: {json.loads(response)["sources"]}')

Answer: "Minyak Sawit Keluarga (Palm Oil Family)" is an artwork by Olivia Koh. It examines her paternal family’s history of farming practices in rural Southern Malaysia (Peninsula), focusing on domestic-scale palm oil plantations in Johor. The work includes footage of these plantations, visual and auditory recordings of funerary Daoist rituals, documents, and interviews that consider the palm oil industry and the relevant history of migration in the local area. It was commissioned by Hyphenated Projects and ACMI. Although it is not currently on display, it was last shown at ACMI: Gallery 5.
Sources: ['https://api.acmi.net.au/works/122462']


In [47]:
query = 'Which works are on display in The Story of the Moving Image?'
response = retrieval_qa.run(query)
print(f'Answer: {json.loads(response)["answer"]}')
print(f'Sources: {json.loads(response)["sources"]}')

Answer: The work titled "Solider's dream effect slide" is currently on display in The Story of the Moving Image → Moving Pictures → MI-02. Play and Illusion section at ACMI: Gallery 1.
Sources: ['https://api.acmi.net.au/works/122524']


In [49]:
query = 'Is there any digital art in the collection?'
response = retrieval_qa.run(query)
print(f'Answer: {json.loads(response)["answer"]}')
print(f'Sources: {json.loads(response)["sources"]}')

Answer: Yes, there is digital art in the collection. The items such as the "Ship and lightning effect slide" and the "Solider's dream effect slide" are examples of digital art. These works are facsimiles created using ultraclear print on acrylic, indicating a digital or modern printing process.
Sources: ['https://api.acmi.net.au/works/122522', 'https://api.acmi.net.au/works/122524']


In [51]:
query = 'Who is Bonnie Cummings?'
response = retrieval_qa.run(query)
print(f'Answer: {json.loads(response)["answer"]}')
print(f'Sources: {json.loads(response)["sources"]}')

Answer: The provided content does not contain any information about a person named Bonnie Cummings.
Sources: ['/content/data.json', 'https://api.acmi.net.au/works/121886', 'https://api.acmi.net.au/works/121889']


In [52]:
query = 'What is "Ship and lightning effect slide" made out of?'
response = retrieval_qa.run(query)
print(f'Answer: {json.loads(response)["answer"]}')
print(f'Sources: {json.loads(response)["sources"]}')

Answer: The "Ship and lightning effect slide" is made out of ultraclear print on acrylic.
Sources: ['https://api.acmi.net.au/works/122522']
